In [4]:
from lambeq import BobcatParser, TreeReader, TreeReaderMode, spiders_reader, cups_reader, stairs_reader
from lambeq import TensorAnsatz, SpiderAnsatz, MPSAnsatz, IQPAnsatz, AtomicType
from discopy import Dim
from classic_pipeline import *
from quantum_pipeline import *

In [2]:
#parser declaration

bobcat_parser = BobcatParser(verbose = "progress")
spider_parser = spiders_reader
cups_parser = cups_reader
stairs_parser = stairs_reader
tree_parser = TreeReader(mode=TreeReaderMode.RULE_ONLY)

In [5]:
#ansatze declaration

tensor_ansatz = TensorAnsatz({AtomicType.NOUN: Dim(2), AtomicType.SENTENCE: Dim(2), AtomicType.CONJUNCTION: Dim(2), AtomicType.PUNCTUATION: Dim(2), AtomicType.NOUN_PHRASE: Dim(2), AtomicType.PREPOSITIONAL_PHRASE: Dim(2)})
spider_ansatz = SpiderAnsatz({AtomicType.NOUN: Dim(2), AtomicType.SENTENCE: Dim(2)})
mps_ansatz = MPSAnsatz({AtomicType.NOUN: Dim(2), AtomicType.SENTENCE: Dim(2)}, bond_dim = 3)
iqp_ansatz = IQPAnsatz({AtomicType.NOUN: 1, AtomicType.SENTENCE: 0}, n_layers = 1, n_single_qubit_params = 3)

In [6]:
#data-extracion for classic pipeline

pip = ClassicPipeline(bobcat_parser, tensor_ansatz)
pip.add_rewriter_rules(ClassicPipeline.SUPPORTED_RULES[0], ClassicPipeline.SUPPORTED_RULES[1], ClassicPipeline.SUPPORTED_RULES[4])
train_labels, train_circuits = pip.create_circuits_and_labels("C:\\Users\\calif\\Documents\\GitHub\\Tesi-Quantum-NLP\\project\\datasets\\edited_datasets\\GPS_edited.csv", "n")
test_labels, test_circuits = pip.create_circuits_and_labels("C:\\Users\\calif\\Documents\\GitHub\\Tesi-Quantum-NLP\\project\\datasets\\edited_datasets\\ePurse_edited.csv", "n")

Parsing tagged sentences:   0%|          | 0/124 [00:00<?, ?it/s]

Parse trees to diagrams:   0%|          | 0/124 [00:00<?, ?it/s]

In [32]:
train_set, test_set = pip.create_dataset(train_labels, train_circuits), pip.create_dataset(test_labels, test_circuits)
pip.create_trainer(train_circuits, test_circuits)
pip.train_model(train_set, test_set)
#pip.plot()

RuntimeError: stack expects each tensor to be equal size, but got [2] at entry 0 and [2, 2] at entry 2

In [ ]:
#data-extracion for quantum pipeline

pip = QuantumPipeline(bobcat_parser, iqp_ansatz)
train_labels, train_circuits = pip.create_circuits_and_labels("code/pipeline/rp_train_data.txt")
test_labels, test_circuits = pip.create_circuits_and_labels("code/pipeline/rp_test_data.txt")